# Выкачивание данных

In [13]:
import sys
import requests
!{sys.executable} -m pip install fake_useragent
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re

In [3]:
import sqlite3
from tqdm.auto import tqdm

In [4]:
conn = sqlite3.connect('voltron.db')
cur = conn.cursor()

In [5]:
# создаю пустую базу

cur.execute("""
CREATE TABLE IF NOT EXISTS main
(id int, title text, author text, date text, kudos int, complete text, age text, genre text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS previews
(id int, preview text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS characters 
(id int, character text) 
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS relationships
(id int, relationship text)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS other_tags 
(id int, tag text) 
""")

conn.commit()  # создаю базу

In [9]:
# conn.close()

In [6]:
session = requests.session()

In [1]:
def article(art):  # обработка одного фанфика
    if not art.find('dd', {'class': 'language'}).text == 'English':
        return None  # выбрасываю всё, что не на английском
    
    fand = art.find('h5', {'class': 'fandoms heading'})
    fand = fand.find_all('a')
    if len(fand) > 1:
        return None  # выбрасываю все кроссоверы, чтобы не множить сущности
    
    marks = art.find_all('span', {'class': 'text'})
    age = marks[0].text
    if age == 'Explicit':
        return None  # выбрасываю очень взрослую литературу, потому что морально не готова обучать модель на таком
    
    f_id = art.find('a')['href']
    f_id = int(re.search('\d+', f_id).group(0))
    
    cur.execute('SELECT id FROM main')
    seen = set(i[0] for i in cur.fetchall())
    if f_id in seen:
        return None
    
    name = art.find('a').text
    auth = art.find('a', {'rel': 'author'})
    if auth:
        auth = auth.text
    else:
        auth = None
    date = art.find('p', {'class': 'datetime'}).text
    kudos = art.find('dd', {'class': 'kudos'})
    if kudos:
        kudos = int(kudos.text)
    else:
        kudos = 0
    # заполняю строку в базе с id, автором, датой публикации, количеством плюшек от читателей,
    # статусом завершён\в процессе\брошен, возрастными ограничениями и (условно) жанром
    
    complete = marks[3].text
    rel = marks[2].text
    cur.execute(
        """
        INSERT INTO main
            (id, title, author, date, kudos, complete, age, genre) 
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (f_id, name, auth, date, kudos, complete, age, rel))
    conn.commit()
    
    relat = [i.text for i in art.find_all('li', {'class': 'relationships'})]
    for r in relat:
        cur.execute(
        """
        INSERT INTO relationships
            (id, relationship) 
            VALUES (?,?)
        """, (f_id, r))
    conn.commit()    
    chars = [i.text for i in art.find_all('li', {'class': 'characters'})]
    for c in chars:
        cur.execute(
        """
        INSERT INTO characters
            (id, character) 
            VALUES (?,?)
        """, (f_id, c))
    conn.commit()
    other = [i.text for i in art.find_all('li', {'class': 'freeforms'})] # + art.find('li', {'class': 'freeforms last'}).text
    for o in other:
        cur.execute(
        """
        INSERT INTO other_tags
            (id, tag) 
            VALUES (?,?)
        """, (f_id, o))
    conn.commit()
    p_text = art.find('blockquote', {'class': 'userstuff summary'})
    if p_text:
        p_text = p_text.text
        
        cur.execute(
        """
        INSERT INTO previews
            (id, preview) 
            VALUES (?,?)
        """, (f_id, p_text))
    else:
        p_text = ''
    conn.commit()
    #  сразу пишу теги + текст превью в текстовый файл для модели
    with open('voltron.txt', 'a', encoding="utf-8") as f:
        add_up = ', '.join(relat+chars+other)
        add_up = add_up + '\n' + p_text + '\n\n'
        f.write(add_up)

In [7]:
def full_page(numba):    #  парсит страницу и находит в ней блоки отдельных фанфиков
    u_a = UserAgent(verify_ssl=False)
    url = f"https://archiveofourown.org/tags/Voltron:%20Legendary%20Defender/works?page={numba}/"
    response = session.get(url, headers={'User-Agent': u_a.random})
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('li', {'role': 'article'})
    for a in articles:
        article(a)

In [14]:
def allmighty(wot):
    for step in tqdm(range(600, wot)):
        full_page(step)

In [15]:
allmighty(2000)

  0%|          | 0/1400 [00:00<?, ?it/s]